In [17]:
from scipy.optimize import minimize, least_squares
import numpy as np
import matplotlib.pyplot as plt
import math

from scipy.interpolate import CubicSpline

"""
theta = the orientation of the cars frame in the world
phi = the steering angle
The configuration of the q = (x, y, theta, phi)
u1 = v_car or the linear speed of the car
u2 = phi_prime, the turning rate of the front wheel

q_prime = f(q, u1, u2)
x_prime = v_car * cos(theta) = u1 * cos(theta)
y_prime = v_car * sin(theta) = u1 * sin(theta)
phi_prime = u2
L = distance between rear and front axles
theta_prime = v_car * 1/L * tan(phi) = u1 * 1/L * tan(phi)
"""

def p(t, tf=10):
    return (10/tf**3) * t**3 - (15/tf**4) * t**4 + (6/tf**5) * t**5

# x(p) = a0 + a1 * p + a2 * p2^2 + a3 * p3^3
# y(p) = b0 + b1 * p + b2 * p2^2 + b3 * p3^3
def opt(x):
    a, b, c, d = x[0], x[1], x[2], x[3]
    #xx = ( (a*d)/2 + (2*b*d)/5 + (c*d)/3 + (d*d)/7 + (2*a*c)/3 + (b*c)/2 + (c*c)/5 + (a*b) + (b*b)/3 + (a*a) )
    xx = ( (9*d*d)/5 + 3*d*c + 2*d*b + (4*c*c)/3 + 2*c*b + b*b )

    a, b, c, d = x[4], x[5], x[6], x[7]
    #yy = ( (a*d)/2 + (2*b*d)/5 + (c*d)/3 + (d*d)/7 + (2*a*c)/3 + (b*c)/2 + (c*c)/5 + (a*b) + (b*b)/3 + (a*a) )
    yy = ( (9*d*d)/5 + 3*d*c + 2*d*b + (4*c*c)/3 + 2*c*b + b*b )

    return xx + yy

# def theta(p_out, x):
#     x_der = (x[2] + 2 * x[3] * p_out + 3 * x[4] * p_out**2)**2
#     y_der = (x[5] + 2 * x[6] * p_out + 3 * x[7] * p_out**2)**2
#     return math.atan2(y_der, x_der)

def x_der(p, x):
    a, b, c, d = x[0], x[1], x[2], x[3]
    der = 3*d*p**2 + 2*c*p + b
    return der

def y_der(p, x):
    a, b, c, d = x[4], x[5], x[6], x[7]
    der = 3*d*p**2 + 2*c*p + b
    return der

def derivative(p, x):
    a, b, c, d = x[0], x[1], x[2], x[3]
    x_der = 3*d*p**2 + 2*c*p + b
    a, b, c, d = x[4], x[5], x[6], x[7]
    y_der = 3*d*p**2 + 2*c*p + b
    return x_der, y_der

# k_0, k_f = 1

# dx/dp = a1 + 2*a2*p + 3*a3*p**2 at p 0 == k_0
    # a1 + 2*a2*0 + 3*a3*0**2 = 1 or k_0
    # a1 = 1 or k_0

# dx/dp = a1 + 2*a2*p + 3*a3*p**2 at p 1 == k_f
    # a1 + 2*a2*1 + 3*a3*1**2 = 1 or k_f
    # 
    # a1*1 + a2*1 + a3*1 = k_f - k_0
    #   a1*1 + a2*1 + a3*1 = 0
    #   a1*1 + a2*1 = -a3

# dy/dp at p 0 == k_0 * tan(theta_0)
    # b0 + b1*0 + b2*0 + b3*0 = 1 * tan(theta_0)
    # b0 = tan(theta_0)

# dy/dp at p 1 == k_f * tan(theta_f)
    # b0 + b1*1 + b2*1 + b3*1 = 1 * tan(theta_f)

# a0 = x0
# a1 = -2*a2*p - 3*a3*p^2
# a2 = 3*a3*p
# 3*a3 = 0
# 
#

def l2(x: np.ndarray):
    a, b, c, d = x[0], x[1], x[2], x[3]
    x_0 = a + b*0 + c*0**2 + d*0**3
    x_f = a + b*1 + c*1**2 + d*1**3

    a, b, c, d = x[4], x[5], x[6], x[7]
    y_0 = a + b*0 + c*0**2 + d*0**3
    y_f = a + b*1 + c*1**2 + d*1**3
    return np.sqrt((x_f - x_0)**2 + (y_f - y_0)**2)

def theta(p_out, x):
    a, b, c, d = x[0], x[1], x[2], x[3]
    x_der = 3*d*p_out**2 + 2*c*p_out + b
    a, b, c, d = x[4], x[5], x[6], x[7]
    y_der = 3*d*p_out**2 + 2*c*p_out + b
    return y_der / x_der

start_x, start_y = 0, 0
#end_x, end_y = 1.8288, 0
end_x, end_y = 0, 2
#end_x, end_y = 5.31114, 0
start_theta, end_theta = 0, 0



# start_theta_rads = math.radians(start_theta)
# end_theta_rads = math.radians(start_theta)
# L = 2
# l1 = math.tan(start_theta_rads)
# l2 = (l1 * (1/math.cos(start_theta_rads))**3) / (2 * L)
# l3 = math.tan(end_theta_rads)
# x_delta = end_x - start_x
# y_delta = end_y - start_y

# a0 = start_x
# a1 = 1
# a2 = l2 / (l3 - l1 + 0.000001) * a1**2 - 2*a1 + (3*(x_delta*l3 - y_delta)) / (l3 - l2 + 0.000001)
# a3 = x_delta - a1 - a2
# b0 = start_y
# b1 = l1 * a1
# b2 = l2*a1**2 + l1*a1
# b3 = y_delta - l2*a1**2 - l1*a1 - l1*a2

#coefs = [1, 1, 1.3, 0.7, 1, 1.9, 1.9, 1.2]
coefs = np.random.random(8)
#x0 = [a0, a1, a2, a3, b0, b1, b2, b3]
#x0 = np.full((8,), 0.00001)
cons = (
        # x(0)
        {'type': 'eq', 'fun': lambda x: x[0] - start_x},
        # x(1)
        {'type': 'eq', 'fun': lambda x: (x[0] + x[1] + x[2]**2 + x[3]**3) - end_x}, 
        # y(0)
        {'type': 'eq', 'fun': lambda x: x[4] - start_y}, 
        # y(1)
        {'type': 'eq', 'fun': lambda x: (x[4] + x[5] + x[6]**2 + x[7]**3) - end_y},
        # dx/dp at p = 0
        #{'type': 'eq', 'fun': lambda x: x_der(0, x) - 1},
        # dx/dp at p = 1
        #{'type': 'eq', 'fun': lambda x: x_der(1, x) - 1},
        # dy/dp at p = 0
        #{'type': 'eq', 'fun': lambda x: y_der(0, x) - math.tan(math.radians(start_theta))},
        # dy/dp at p = 1
        #{'type': 'eq', 'fun': lambda x: y_der(1, x) - math.tan(math.radians(end_theta))},
        
        #{'type': 'eq', 'fun': lambda x: x[5] - math.tan(math.radians(start_theta))},
        #{'type': 'eq', 'fun': lambda x: theta(0, x) - math.tan(math.radians(start_theta))},
        #{'type': 'eq', 'fun': lambda x: theta(0, x) - math.tan(math.radians(start_theta))},
        
        
        #{'type': 'eq', 'fun': lambda x: x[5] + 2 * x[6] + 3 * x[7] - math.tan(math.radians(end_theta))}
        #{'type': 'eq', 'fun': lambda x: theta(1, x) - math.tan(math.radians(end_theta))}
    )
# Try generating the cubic spline with control points to enforce the starting 
# and ending positions' thetas
cs = CubicSpline(x=[start_x, start_x + 0.05, end_x - 0.05, end_x], y=[start_y, start_y, end_y, end_y])
#cs = CubicSpline(x=[start_x, end_x], y=[start_y, end_y])
xs = np.arange(0, end_x, 0.01)
fig, ax = plt.subplots(figsize=(6.5, 4))
ax.plot(xs, cs(xs), label="S")
plt.show()

assert False
res = minimize(l2, coefs, tol=1e-6, constraints=cons)
B = res.x
xs = []
ys = []
points = []
num_points = 30
print(B)
for i in range(num_points):
    p_out = p(i, tf=num_points)
    x = B[0] + B[1] * p_out + B[2] * p_out**2 + B[3] * p_out**3
    y = B[4] + B[5] * p_out + B[6] * p_out**2 + B[7] * p_out**3
    if y < 0.0000001:
        y = 0
    #points.append([x / (end_x - start_x), y / (end_y - start_y)])
    points.append([x, y])
    xs.append(x)
    ys.append(y)
#print((end_y - start_y) / (end_x - start_x))
print(points)
#print(math.degrees(theta(1, B)))
#rint(math.degrees(theta(0, B)))
plt.plot(xs, ys)
#print(res.x)


ValueError: `x` must be strictly increasing sequence.

In [26]:
# Should ONLY be 3D points that have been seen in all the previous and current frames
def windowed_bundle_adjustment(pts_2d, pts_3d, cam_poses):
    def loss(current_pose: np.ndarray):
        error = 0
        er = lambda twod, threed, pose: np.sum((twod - (pose.dot(threed) / threed[2])[:2])**2)
        reshaped = current_pose.reshape((3, 4))
        for k in range(len(cam_poses) - 1):
            error += er(pts_2d[k][0], pts_3d[k][0], reshaped)
            error += er(pts_2d[k][1], pts_3d[k][1], reshaped)
            error += er(pts_2d[k][2], pts_3d[k][2], reshaped)
        # error += er(pts_2d[-1][0], pts_3d[-1][0], reshaped)
        # error += er(pts_2d[-1][1], pts_3d[-1][1], reshaped)
        # error += er(pts_2d[-1][2], pts_3d[-1][2], reshaped)
        return error
    return np.reshape(minimize(loss, cam_poses[-1].flatten()).x, (3, 4))

def create_3d_point3(camera_pose: np.ndarray):
    pts_2d, pts_3d = [], []
    for i in range(3):
        pt_3d = np.random.random((4,))
        pt_2d = (camera_pose.dot(pt_3d) / pt_3d[2])[:2] #+ np.random.random() * 2
        pts_2d.append(pt_2d)
        pts_3d.append(pt_3d)
    return pts_2d, pts_3d

first_pose = np.zeros((3, 4))
first_pose[0, 0] = 1
first_pose[1, 1] = 1
first_pose[2, 2] = 1
first_pose[2, 3] = 1
first_2d, first_3d = create_3d_point3(first_pose)
second_pose = np.zeros((3, 4))
second_pose[0, 0] = 1
second_pose[1, 1] = 1
second_pose[2, 2] = 1
second_pose[2, 3] = 1
second_pose[:, 3] += 10
second_2d, second_3d = create_3d_point3(second_pose)
print(f"Actual Projection Matrix:\n {second_pose}\n")
second_pose += np.random.random((3, 4)) * 0.1
print(f"Noisy Projection Matrix:\n {second_pose}\n")
res = windowed_bundle_adjustment([first_2d, second_2d], [first_3d, second_3d], [first_pose, second_pose])
print(f"Optimized Projection Matrix:\n {res}\n")


Actual Projection Matrix:
 [[ 1.  0.  0. 10.]
 [ 0.  1.  0. 10.]
 [ 0.  0.  1. 11.]]

Noisy Projection Matrix:
 [[1.00785993e+00 1.87709472e-02 3.14075516e-02 1.00642556e+01]
 [6.54531321e-02 1.09902441e+00 6.88818989e-03 1.00188949e+01]
 [8.16722473e-02 8.81792980e-02 1.00166019e+00 1.10795428e+01]]

Optimized Projection Matrix:
 [[-0.78373097 -1.14715875 -1.00358933  9.47609231]
 [-1.7725235  -0.1398736  -0.99739267  9.41643068]
 [ 0.08167225  0.0881793   1.00166019 11.07954284]]

